# MAPLES-DR Visualisation Example


In [1]:
# Import maples-dr
import cv2
import maples_dr
import numpy as np

# Import visualization tools
from ipywidgets import HTML, GridBox, Layout
from jppype import View2D, imshow, sync_views, vscode_theme
from maples_dr.dataset import BiomarkerField as Biomarkers
from maples_dr.dataset import FundusField as Fundus

vscode_theme()

Thanks for using MAPLES-DR!
  When using this dataset in academic works,
  please cite: ]8;id=825925;https://arxiv.org/abs/2402.04258\https://arxiv.org/abs/2402.04258]8;;\

HTML(value="<style>\n        .cell-output-ipywidget-background {\n                background: transparent !imp…

## Configure Maples-DR


Configure local path, resolution and image format.


In [2]:
maples_dr.configure(
    maples_dr_path="PATH/TO/MAPLES-DR/AdditionalData.zip",
    messidor_path="/home/gaby/These/Data/Fundus/MESSIDOR1500/0-images/",
    image_format="bgr",
    preprocessing="none",
)

Output()

Extracted 0 images in 0.0 second.

## Load sample


Load the train set in memory.


In [3]:
dataset = maples_dr.quick_api.GLOBAL_LOADER.load_dataset("all_with_duplicates")

Read the first image and its corresponding labels.


In [9]:
ID = 5
no_cup = [
    "20060412_59636_0200_PP",
    "20060411_57853_0200_PP",
    "20060411_59919_0200_PP",
    "20060412_58723_0200_PP",
    "20060411_61060_0200_PP",
    "20060411_60283_0200_PP",
]
sample = dataset[no_cup[ID]]

## Show sample image and labels.


Display the anatomical biomarkers using `jppype.View2D` widget.

_(Scroll in the views to zoom in or out. Press **v** after clicking on a view to toggle biomarkers visibility.)_


In [10]:
def header(title, legend=None):
    style = "color: var(--jppype-foreground-color); font-size: 20px; text-align: center;"
    header = f'<h2 style="{style}">{title}</h2>'
    if legend is not None:
        header += '<p style="color: var(--jppype-foreground-color); font-size: 14px; text-align: center;">'
        for key, value in legend.items():
            header += (
                f'<span style="display: inline-block"><span style="background-color: {value};'
                ' width: 14px; height: 14px; border-radius: 7px; display: inline-block; margin-left: 20px">'
                f"</span> {key} </span>"
            )
        header += "</p>"
    return HTML(header)


def add_label(view, label, name, colormap=None):
    color = None if colormap is None or name not in colormap else colormap[name]
    view.add_label(label, name, opacity=0.8, colormap=color)


legend1 = {
    "ROI Mask": "#aaa",
    "Optic Disc": "#767fab",
    "Optic Cup": "#7aa8ba",
    "Macula": "#640070",
    "Vessels": "#a56fb9",
}
h1 = header("Anatomical Structures", legend1)
v1 = imshow(sample[Fundus.FUNDUS])

legend2 = {
    "Exudates": "#334f7b",
    "Cotton Wool Spots": "#7aa8ba",
    "Drusens": "#4c859b",
    "Bright Uncertains": "#ddd",
}
h2 = header("Bright Lesions", legend2)
v2 = imshow(sample[Fundus.FUNDUS])
v2._left_ruler = False

legend3 = {
    "Microaneurysms": "#490054",
    "Hemorrhages": "#783986",
    "Neo-vessels": "#a893b6",
    "Red Uncertains": "#ddd",
}
h3 = header("Red Lesions", legend3)
v3 = imshow(sample[Fundus.FUNDUS])
v3._left_ruler = False


def setup_view(sample):
    v1["background"] = sample[Fundus.FUNDUS]
    add_label(v1, 1 - sample[Fundus.MASK], "ROI Mask", legend1)
    add_label(v1, sample[Biomarkers.VESSELS], "Vessels", legend1)
    add_label(v1, sample[Biomarkers.OPTIC_DISC], "Optic Disc", legend1)
    add_label(v1, sample[Biomarkers.OPTIC_CUP], "Optic Cup", legend1)
    add_label(v1, sample[Biomarkers.MACULA], "Macula", legend1)

    v2["background"] = sample[Fundus.FUNDUS]
    add_label(v2, sample[Biomarkers.BRIGHT_UNCERTAINS], "Bright Uncertains", legend2)
    add_label(v2, sample[Biomarkers.COTTON_WOOL_SPOTS], "Cotton Wool Spots", legend2)
    add_label(v2, sample[Biomarkers.EXUDATES], "Exudates", legend2)
    add_label(v2, sample[Biomarkers.DRUSENS], "Drusens", legend2)

    v3["background"] = sample[Fundus.FUNDUS]
    add_label(v3, sample[Biomarkers.NEOVASCULARIZATION], "Neo-vessels", legend3)
    add_label(v3, sample[Biomarkers.RED_UNCERTAINS], "Red Uncertains", legend3)
    add_label(v3, sample[Biomarkers.HEMORRHAGES], "Hemorrhages", legend3)
    add_label(v3, sample[Biomarkers.MICROANEURYSMS], "Microaneurysms", legend3)


setup_view(sample)
sync_views(v1, v2, v3)

GridBox(
    [h1, h2, h3, v1, v2, v3],
    layout=Layout(grid_template_columns="repeat(3, 1fr)", grid_template_rows="auto 600px"),
)

GridBox(children=(HTML(value='<h2 style="color: var(--jppype-foreground-color); font-size: 20px; text-align: c…

In [6]:
maples_dr.configure(resize=True)
v = View2D()
v.add_label(
    dataset["20060411_61478_0200_PP"]["vessels"] + 1,
    colormap={1: "#F3EEF6", 2: "#AB80BA"},
)
GridBox(
    [v],
    layout=Layout(grid_template_columns="1fr", height="600px"),
)

GridBox(children=(View2D(),), layout=Layout(grid_template_columns='1fr', height='600px'))

In [7]:
av_img = cv2.imread("/home/gaby/These/Data/Fundus/MAPLES-DR/AV_reviewed/20060412_61501_0200_PP.png")
av_map = np.zeros(av_img.shape[:2], dtype=np.uint8)
av_map[av_img[:, :, 1] > 125] = 3
av_map[av_img[:, :, 0] > 125] = 2
av_map[av_img[:, :, 2] > 125] = 1

v = imshow(dataset["20060412_61501_0200_PP"]["fundus"])
v.add_label(av_map, colormap={1: "red", 2: "blue", 3: "green"})
v

View2D()